In [ ]:
# FEniCSx version of continuum learning simulation

import numpy as np
import matplotlib.pyplot as plt

from mpi4py import MPI
from dolfinx import mesh, fem, plot
from dolfinx.fem import Function, functionspace, dirichletbc, locate_dofs_geometrical, Constant
import dolfinx.fem.petsc
from ufl import TrialFunction, TestFunction, dot, grad, dx, sqrt, grad
from basix.ufl import element
from petsc4py import PETSc

In [ ]:
# --- Parameters ---
mod = 1
num_tasks = 1
save_to_table = False
iterations = 6
if_periodic = False

x_min, x_max = -0.5, 0.5
y_min, y_max = -2.5, 2.5

alpha = 1.0
beta = 0.0
gamma = 0.15
y1 = y2 = 1.5
yout1 = yout2 = -1.5
sigma = 0.05
sigma2 = 0.1

In [ ]:
# --- Mesh ---
domain = mesh.create_rectangle(MPI.COMM_WORLD,
                               [[x_min, y_min], [x_max, y_max]],
                               [46, 46],
                               mesh.CellType.triangle)

coords = domain.geometry.x

In [ ]:
# --- projection functions ---

def project_scalar_func(func):
    u = TrialFunction(ScalarFuncSpace)
    v = TestFunction(ScalarFuncSpace)    
    a_proj = fem.form(dot(u, v) * dx)
    L_proj = fem.form(dot(func, v) * dx)     
    proj_func = fem.Function(ScalarFuncSpace)
    proj_problem = fem.petsc.LinearProblem(a_proj, L_proj, u=proj_func)
    proj_func = proj_problem.solve()
    return proj_func

def project_vector_func(func):
    U = TrialFunction(VectorFuncSpace)
    V = TestFunction(VectorFuncSpace)    
    a_proj = fem.form(dot(U, V) * dx)
    L_proj = fem.form(dot(func, V) * dx)     
    proj_func = fem.Function(VectorFuncSpace)
    proj_problem = fem.petsc.LinearProblem(a_proj, L_proj, u=VectorFuncSpace)
    proj_func = proj_problem.solve()
    return proj_func    

def Poisson_measurement(left_bc, right_bc, c):
    # Weak form: a = (c * grad(u), grad(v)) * dx
    a = fem.form(dot(c * grad(u), grad(v)) * dx)
    L = fem.form(Constant(domain, 0.0) * v * dx)

    # Input BC (left)
    inval_fn = Function(ScalarFuncSpace)
    inval_fn.interpolate(left_bc)
    left_dofs = locate_dofs_geometrical(ScalarFuncSpace, lambda x: np.isclose(x[0], x_min))
    bc_left = dirichletbc(inval_fn, left_dofs)

    # Output BC (right) – placeholder for now
    outval_fn = Function(ScalarFuncSpace)
    outval_fn.interpolate(right_bc)
    right_dofs = locate_dofs_geometrical(ScalarFuncSpace, lambda x: np.isclose(x[0], x_max))
    bc_right = dirichletbc(outval_fn, right_dofs)

    # both bcs
    bcs = [bc_left, bc_right]

    # Solve Poisson equation
    p_sol = Function(ScalarFuncSpace)
    problem = fem.petsc.LinearProblem(a, L, bcs=bcs, u=p_sol)
    p_sol = problem.solve()
    return p_sol

def calculate_Q(p):
    U = TrialFunction(VectorFuncSpace)
    V = TestFunction(VectorFuncSpace)
    a_proj = fem.form(dot(U, V) * dx)
    L_proj = fem.form(dot(grad(p)*c, V) * dx)
    Q = fem.Function(VectorFuncSpace)      
    problem = fem.petsc.LinearProblem(a_proj, L_proj, u=Q)
    Q = problem.solve()
    return Q
    
def calculate_gradc(c):
    U = TrialFunction(VectorFuncSpace)
    V = TestFunction(VectorFuncSpace)
    a_proj = fem.form(dot(U, V) * dx)
    L_proj = fem.form(dot(grad(c), V) * dx)
    grad_c = fem.Function(VectorFuncSpace)      
    problem = fem.petsc.LinearProblem(a_proj, L_proj, u=Q)
    grad_c = problem.solve()
    return grad_c

In [ ]:
# --- plot functions ---

def plot_scalar_field(field, string):
    plt.figure(figsize=(6, 4))
    plt.tricontourf(coords[:, 0], coords[:, 1], field.x.array, levels=100, cmap="plasma")
    plt.title(f"{string} at iteration {i+1}")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.colorbar(label=string)
    plt.tight_layout()
    plt.show()

In [ ]:
# --- Function space ---
ScalarFuncSpace = functionspace(domain, ("Lagrange", 1))
VectorFuncSpace = functionspace(domain, ("CG", 1, (2,)))

# --- Trial and Test Functions ---
u = TrialFunction(ScalarFuncSpace)
v = TestFunction(ScalarFuncSpace)
U = TrialFunction(VectorFuncSpace)
V = TestFunction(VectorFuncSpace)

# --- Input/target functions ---
def input_val(y):
    return [np.exp(-(y - y1)**2 / sigma) + 0.5,
            np.exp(-(y - y2)**2 / sigma) + 0.5]

def target_val(y):
    return [1.0 * np.exp(-(y - yout1)**2 / sigma) + 0.75,
            1.0 * np.exp(-(y - yout2)**2 / sigma) + 0.75]

# --- Plot input and target ---
ys = np.linspace(y_min, y_max, 400)
plt.plot(ys, [input_val(y)[0] for y in ys], 'b', label='Input')
plt.plot(ys, [target_val(y)[0] for y in ys], 'r', label='Target')
plt.title('Input and Target Iteration 1')
plt.xlabel('y')
plt.ylabel('Value')
plt.legend()
plt.show()

In [ ]:
# --- Dynamics Loop ---
for j in range(num_tasks):
    # Initialize conductivity field c
    c = Function(ScalarFuncSpace)
    c.interpolate(lambda x: np.full(x.shape[1], 1.0))

    loss_lst = []

    for i in range(iterations):
        k = i % mod

        left_bc = lambda x: np.exp(-(x[1] - y1)**2 / sigma) + 0.5
        right_bc = lambda x: np.zeros(x.shape[1])
        p_sol = Poisson_measurement(left_bc, right_bc, c)

        # Plot p_sol
        plt.figure(figsize=(6, 4))
        plt.tricontourf(coords[:, 0], coords[:, 1], p_sol.x.array, levels=100, cmap="viridis")
        plt.title(f"Pressure at iteration {i+1}")
        plt.xlabel("x")
        plt.ylabel("y")
        plt.colorbar(label="Pressure")
        plt.tight_layout()
        plt.show()

        # --- Compute gradients ---
        
        # grad_p = fem.Function(VectorFuncSpace)
        # grad_p.interpolate(lambda x: np.vstack(np.gradient(p_sol.x.array.reshape((46, 46)))))
        # # Compute flux Q = -c * grad(p)
        # Q = fem.Function(VectorFuncSpace)
        # Q.interpolate(lambda x: -c.eval(x).T * grad_p.eval(x).T)
        # absQ = fem.Function(ScalarFuncSpace)
        # absQ.interpolate(lambda x: np.sqrt(np.sum(Q.eval(x)**2, axis=0)))

        # grad_c = fem.Function(V_vec)
        # grad_c.interpolate(lambda x: np.vstack(np.gradient(c.x.array.reshape((46, 46)))))

        # absGradc = fem.Function(ScalarFuncSpace)
        # absGradc.interpolate(lambda x: np.sqrt(np.sum(grad_c.eval(x)**2, axis=0)))

        # absGradc_qtr = fem.Function(ScalarFuncSpace)
        # absGradc_qtr.interpolate(lambda x: absGradc.eval(x)**0.25)
        
        # calculate Q
        Q = calculate_Q(p_sol)

        # calculate |Q|
        absQ_expr = sqrt(dot(Q, Q))
        absQ = project_scalar_func(absQ_expr)

        # plot |Q|
        plot_scalar_field(absQ, "Q")

        # Compute |∇c|^0.25
        grad_c = calculate_gradc(c)
        abs_grad_c_pow_expr = sqrt(dot(Q, Q))**(1/4)
        abs_grad_c_pow = project_scalar_func(abs_grad_c_pow_expr)

        # plot |∇c|^0.25
        plot_scalar_field(abs_grad_c_pow, "|∇c|^0.25")

         # --- Update conductivity ---
        c_new = project_scalar_func(gamma * absQ - beta * abs_grad_c_pow + 1.0)
        plot_scalar_field(c_new, "c")
        # c_new = fem.Function(ScalarFuncSpace)
        # c_new.interpolate(lambda x: gamma * absQ.eval(x) - beta * absGradc_qtr.eval(x) + 1.0)
        # c.interpolate(lambda x: c_new.eval(x))
        c = c_new


In [ ]:
import fe
fe.plot(p_sol)

In [ ]:
plt.tricontourf(coords[:, 0], coords[:, 1], grad(p_sol).x.array, levels=100, cmap="plasma")